In [11]:
import pandas as pd
import os
import string
import re
from nltk.stem import WordNetLemmatizer
import nltk
import pickle
print('NLTK (DOWNLOAD ALL PACKAGES TO PERFORM NLP OPERATION)')

print('UNCOMMENT FOLLOWING LINE To GET NLTK DOWNLOADED')
#nltk.download()
stopword = nltk.corpus.stopwords.words('english')
wordnet_lemmatizer = WordNetLemmatizer()

NLTK (DOWNLOAD ALL PACKAGES TO PERFORM NLP OPERATION)
UNCOMMENT FOLLOWING LINE To GET NLTK DOWNLOADED


In [12]:
def preprocess(df):
    
    df = df[df.columns.drop(list(df.filter(regex='^Cat')))]
    df = df[df['Date'] != '27/06/2001']  #removing the date
    df[(df['Subject'] != 'RE:') & (df['Subject'] != 'FW:') & (df['Subject'] != 'Re:')]  #removing the max same subjects
    del df['Unnamed: 0']
    return df

In [13]:
def clean_text(text):
    text_nopunct = "".join([char for char in text.lower() if char not in string.punctuation])
    tokens = re.split('\W+', text_nopunct) #tokenize
    words_without_stopwords = [word for word in tokens if word not in stopword] #remove stopwords from tokens
    return [wordnet_lemmatizer.lemmatize(word, pos="v") for word in words_without_stopwords]

In [14]:
DATA = 'data' #https://data.world/brianray/enron-email-dataset

FILENAMES = [os.path.join(DATA, filename) for filename in os.listdir(DATA)]
df = pd.read_csv(FILENAMES[0])
df = preprocess(df)

In [15]:
df.shape

(100000, 15)

In [16]:
df[df['Subject'].isna()].shape

(5364, 15)

In [17]:
df[df['content'].isna()].shape

(402, 15)

##### Right now just removing the nan value, may look into this later for do preprocessing

In [18]:
df = df[df['Subject'].notna()]
df = df[df['content'].notna()]
df[df['content'].isna()].shape

(0, 15)

In [19]:

dfEmail = df[['Subject', 'content']]
# dfEmail['content_lemma'] = dfEmail['content'].apply(lambda x: clean_text(x))
# dfEmail['Subject_lemma'] = dfEmail['content'].apply(lambda x: clean_text(x))

In [20]:
dfEmail.head()

,Subject,content
1,Re:,Traveling to have a business meeting takes the...
2,Re: test,test successful. way to go!!!
4,Re: Hello,Let's shoot for Tuesday at 11:45.
5,Re: Hello,"Greg, How about either next Tuesday or Thursda..."
7,Re: PRC review - phone calls,any morning between 10 and 11:30


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
vectorizer = TfidfVectorizer(stop_words='english', analyzer=clean_text)

In [24]:
vectorizer.fit(dfEmail['content'])
print("\n Feature names Identified :\n")
print(vectorizer.get_feature_names())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [27]:
print(len(vectorizer.get_feature_names()))

304525


In [28]:
#saveing content vectorizer for future use
pickle.dump(vectorizer, open("tfidf_content.pickle", "wb"))

simillarly, we will do the same on subject

In [29]:
vectorizer_subject = TfidfVectorizer(stop_words='english', analyzer=clean_text)
vectorizer_subject.fit(dfEmail['Subject'])
print("\n Feature names Identified :\n")
print(vectorizer_subject.get_feature_names())
print(len(vectorizer_subject.get_feature_names()))
#saveing content vectorizer for future use
pickle.dump(vectorizer_subject, open("tfidf_subject.pickle", "wb"))


 Feature names Identified :

['', '0', '00', '000', '00000000000000000000000000000000000weil', '000000000055936', '000000000078904', '0000105674', '0000108727', '0000110257', '0000110295', '0000110326', '0000110685', '0000264227478000012', '0002001', '0008002', '0008037', '001', '0010002', '0010028', '0010038', '001018xls', '0011001', '00110100095649', '0011038', '0011056', '00240833807905653', '00262869908194660', '003', '0048', '0060', '0074', '00ceovol1', '01', '010', '0100', '010000004', '010001522', '010102', '010202', '010402', '010501', '0105064', '01070111', '010702', '01074', '01082002', '010902', '01092002', '011101', '011115', '01140118', '011402', '01142002', '011442076824251', '0115', '01152002', '01154', '01155', '01158', '011602', '01162', '01162002', '01164', '011702', '01172', '01172002', '01174', '01176', '01177', '01178', '0118', '01181', '01182002', '01186', '01187', '01188', '011901', '01210125', '012201', '012202', '012301', '01235', '01236', '01237', '01238pdf',

In [31]:
data = vectorizer.transform(['Traveling to have a business meeting takes th'])

In [32]:
print(data)

  (0, 281433)	0.5627821670824664
  (0, 275644)	0.5928984666649425
  (0, 272174)	0.31870101597955874
  (0, 197960)	0.31271587959143815
  (0, 88537)	0.363848982003898


In [33]:
data = vectorizer_subject.transform(['Traveling to have a business meeting takes th'])
print(data)

  (0, 16521)	0.4689750455097545
  (0, 16200)	0.5972249190605863
  (0, 15961)	0.46171075886038737
  (0, 10933)	0.2471234667974454
  (0, 4164)	0.3861838552461468
